In [1]:
#importamos las librerias
import os
import pandas as pd
import re

In [2]:
#Vamos a instalar unas libreria, especificamente la libreria transformers de "huggingface"
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 34.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.1 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
#importamos la libreria de transformers
import transformers
#y multiples clasificadores
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [4]:
#Importamos la libreria para hacer el split train/test
from sklearn.model_selection import train_test_split

In [5]:
# Importamos las librerias de pytorch
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [6]:
# Bajamos los datos
!curl -O http://srodriguez.me/Datasets/imdb.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.7M  100 25.7M    0     0  2234k      0  0:00:11  0:00:11 --:--:-- 1978k


In [7]:
#Los descomprimimos
!unzip imdb.zip

Archive:  imdb.zip
  inflating: IMDB Dataset.csv        


In [8]:
#Leemos el DataFrame
df = pd.read_csv("IMDB Dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
#mostramos los primeros 5
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
#Muestreamos 10.000 reviews al azar
df = df.sample(10000)
df.reset_index(drop=True,inplace=True)

In [11]:
# Transformamos el output en valores numericos
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [12]:
df.head()

,review,sentiment
0,"""COSBY,"" in my opinion, is a must-see CBS hit!...",1
1,"This was the worst movie I've ever seen, yet i...",1
2,I am a huge Ziyi Zhang fan and will go to any ...,0
3,Just Cause is one of those films that at first...,0
4,The untold origin of the Lone Ranger. It shows...,0


In [13]:
#Mostramos la primer review
df.review[0]

'"COSBY," in my opinion, is a must-see CBS hit! I\'m not sure if I\'ve never seen every episode, but I still enjoyed it. It\'s hard to say which one is my favorite. Also, I really loved the theme song. If you ask me, even though I liked everyone, it would have been nice if Madeline Kahn hadn\'t passed away during the show\'s run. Since that happened, I\'ve always wondered what the show would have been like. Everyone always gave a good performance, the production design was spectacular, the costumes were well-designed, and the writing was always very strong. In conclusion, even though it can be seen on TBS now, I strongly recommend you catch it just in case it goes off the air for good'

In [14]:
#Obtenemos set de train, validacion y testing
temp_df, test_df = train_test_split(df,test_size=0.3,random_state=42)
train_df, val_df = train_test_split(temp_df,test_size=0.1,random_state=42)

train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [15]:
val_df

,review,sentiment
0,"Amateur, no budget films can be surprisingly g...",0
1,By 1941 Columbia was a full-fledged major stud...,0
2,Did Francis Ford Coppola have a brain aneurysm...,0
3,Origins of the Care Bears & their Cousins. If ...,1
4,<br /><br />Arriving by boxcar in New York Cit...,1
...,...,...
695,Dire beyond belief. Obviously set on the Isle ...,0
696,One of Fuller's (a combat veteran himself) ear...,1
697,This film almost gave me a nervous breakdown. ...,0
698,I admit to being in awe of the Sea and have sp...,1


In [16]:
lengths = train_df.review.apply(lambda x: len(x.split(" ")))
lengths.max()

2108

In [17]:
#Cargamos un modelo de bert y su tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
#Hagamos una prueba de como bert tokeniza una oración
tokenizer.tokenize("Hi, this is a test to see how Bert tokenizer processes a sentence")

['hi',
 ',',
 'this',
 'is',
 'a',
 'test',
 'to',
 'see',
 'how',
 'bert',
 'token',
 '##izer',
 'processes',
 'a',
 'sentence']

In [19]:
#El vocabulario no es necesario ajustarlo, ya viene pre-entrenado y po lo mismo, 
#La transformación a indices númericos se utiliza con el metodo encode plus
tokenizer.encode_plus("Hi, this is a test to see how Bert tokenizer processes a sentence")

{'input_ids': [101, 7632, 1010, 2023, 2003, 1037, 3231, 2000, 2156, 2129, 14324, 19204, 17629, 6194, 1037, 6251, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
#Vamos a generar un CustomDataset (una formar de cargar datos en batches)
#Para pytorch, nos va a facilitar el entrenamiento
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.sentence = dataframe.review.values
        self.targets = self.data.sentiment.values
        self.max_len = max_len
    def __len__(self):
        return len(self.sentence)
    def __getitem__(self, index):
        sentence = str(self.sentence[index])
        sentence = " ".join(sentence.split())
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids'] #Ids del vocabulario
        mask = inputs['attention_mask'] #Mascaras para definir donde la atencion deberia ver
        token_type_ids = inputs["token_type_ids"] #requerido pero no utilizado
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [21]:
#Generamos nuestra conexion a la GPU
from torch import cuda
device = 'cuda:0' if cuda.is_available() else 'cpu'

In [22]:
device

'cuda:0'

In [23]:
# Definimos constantes para la ejecucion de nuestro codigo
MAX_LEN = 200 # 2470
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-05

In [24]:
#Transformamos los df en datasets
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
val_set = CustomDataset(val_df, tokenizer, MAX_LEN)
test_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [25]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

#Luego generamos los dataloaders de pytorch

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(test_set, **test_params)

In [33]:
#Vamos a definir una clase de BERT modificada
class BERTClass(torch.nn.Module):
    def __init__(self,b_model):
        super(BERTClass, self).__init__()
        self.l1 = b_model # Definimos que la primera capa sea el modelo de bert que recibe por parametro
        self.l2 = torch.nn.Dropout(0.1) # A la salida de bert, vamos a aplicar un dropout de 0.1
        self.l3 = torch.nn.Linear(768, 1) #Finalmente vamos a definir una capa densa (combinación lineal) de la dimension de bert a nuestra neurona de salida
        self.out_activation = torch.nn.Sigmoid() # función de activación (que no vamos a utilizar)
    
    def forward(self, ids, mask, token_type_ids):
        #Hacemos el forward pass por BERT, dropout y la capa densa
        output = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids) 
        output_2 = self.l2(output[1])
        output = self.l3(output_2)
        #output = self.out_activation(output)
        return output

#Generamos un modelo de BERTClass
model = BERTClass(bert_model)
#llevamos el modelo a la GPU
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [34]:
#Definimos nuestra funcion de perdida
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [35]:
#El optimizador
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [36]:
from tqdm.notebook import tqdm

In [37]:
#Definimos nuestro loop de entrenamiento
def train(epoch):
    
    model.train()
    loss_acum=0
    for iters , data in tqdm(enumerate(training_loader, 0), total = len(training_loader)):
        #print(len(data['ids'][0]))
        #print(len(data['mask'][0]))
        #print(len(data['token_type_ids'][0]))
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        #print(len(ids),len(mask),len(token_type_ids))
        #print(ids,mask,token_type_ids,targets)
        outputs = model(ids, mask, token_type_ids)
        outputs = outputs.squeeze(1)
        #print(outputs,targets)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        loss_acum += loss.item()
        if iters%5000==0:
            print(outputs,targets)
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(iters)
    return loss_acum/iters

In [38]:
#Definimos nuestro loop de validacion/testing
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0),total = len(testing_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).squeeze(1).cpu().detach().numpy())
    
    return np.array(fin_outputs), np.array(fin_targets)

In [ ]:
#iteramos por nuestras epocas
for epoch in range(EPOCHS):
    print(epoch)
    epoch_loss = train(epoch)
    print(f'epoch loss: {epoch_loss}')

0


  0%|          | 0/788 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor([ 0.0140, -0.5342, -0.2073, -0.0148, -0.2679, -0.6079, -0.2351, -0.4106],
       grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 1., 0., 0., 0., 1.])
Epoch: 0, Loss:  0.712881326675415


In [ ]:
Wfrom sklearn import metrics
import numpy as np

#Corremos un loop de testing
for epoch in range(1):
    outputs, targets = validation(epoch)

In [ ]:
#Calculamos los valores binarios y luego obtenemos las metricas de desempeño
outputs_bin = (outputs >= 0.5)
accuracy = metrics.accuracy_score(targets, outputs_bin)
f1_score_micro = metrics.f1_score(targets, outputs_bin, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs_bin, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
print(metrics.classification_report(targets, outputs_bin))